# FurnitureGen: AI-Powered Interior Design

Jeff Barney, MSAI-495: Generative AI Image Project

### Business Goal / Case Statement
Enable fast, iterative interior design by allowing users to vizualize any room in a different style.

### Data
* I downloaded my dataset from [Kaggle](https://www.kaggle.com/datasets/stepanyarullin/interior-design-styles)
* The dataset consists of a collection of interior design images (~1,000 per style) scraped from Houzz.com
* The data set contains 14,876 images in the training set, and 3,729 in the test set
* All images are 320x320 and in the jpg file format

In [ ]:
# enable Notebook to access Google Drive

from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
ASSIGNMENT_PATH = "//content/gdrive/My\ Drive/Kellogg/MSAI\ -\ Gen\ AI/Image\ Project"
%cd $ASSIGNMENT_PATH
%ls -a

In [3]:
# Import the data set into Google Collab
import kagglehub

kagglehub.dataset_download("stepanyarullin/interior-design-styles")

100%|██████████| 699M/699M [00:05<00:00, 139MB/s]

Extracting files...


'/root/.cache/kagglehub/datasets/stepanyarullin/interior-design-styles/versions/1'

### Step 1 - Visualize the data

In [14]:
from preprocessing import sample_data
from IPython.display import HTML

df = sample_data()
HTML(df.to_html(escape=False))

,Label,Image
0,transitional,
1,industrial,
2,shabby-chic-style,
3,asian,
4,victorian,
5,coastal,
6,southwestern,
7,craftsman,
8,contemporary,
9,scandinavian,


### Step 2 - Load and Normalize the Training Data

In [19]:
from preprocessing import fetch_data, preprocess

train_data = fetch_data()
train_data = preprocess(train_data)

### Step 3 - Create an instance of the Diffusion model

In [25]:
from diffusion_model import DiffusionModel
from hyperparameters import LOAD_MODEL, WEIGHT_PATH

ddm = DiffusionModel()
ddm.normalizer.adapt(train_data)

if LOAD_MODEL:
    ddm.built = True
    ddm.load_weights(WEIGHT_PATH)

### Step 4 - Train the model

In [ ]:
from tensorflow.keras import losses, optimizers
from hyperparameters import EPOCHS, LEARNING_RATE, WEIGHT_DECAY
from training_callbacks import model_checkpoint_callback, tensorboard_callback, image_generator_callback

ddm.compile(optimizer=optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY), loss=losses.MeanAbsoluteError())
ddm.fit(train_data, epochs=EPOCHS, callbacks=[
    #model_checkpoint_callback(), Don't save model weights for now, too complicated to manage on Google Drive
    tensorboard_callback(),
    image_generator_callback(),
])

Output hidden; open in https://colab.research.google.com to view.

# Notes

### Output
Output for the final training round that I did on my model can be found in the repo as [image-project-output.pdf.](./image-project-output.pdf) Note that the output is cut off, this is because my Google Colaboratory session was interrupted on epoch 158 after running for a couple hours.

### Development process
Throughout this project, I learned a lot about ML Ops workflows and development. I had an incredibly difficult time getting tensorflow to recognize the GPU on the devices that I would provision through Quest On Demand. After spending many hours on this same issue, I ultimately resorted to using a Google Colaboratory runtime with a T4 GPU and found my training time to be dramatically reduced. Once I had an effective setup in Google Colaboratory where I had access to my dataset and appropriate hardware, I made progress very quickly. 

### Key Learnings
* Python dependency management is a nightmare. Because ML workflows rely on libraries with low-level functionality, such as tensorflow, getting an effective set up can be non-trivial.
* Hardware profiling can produce dramatic speed ups. Although I did not do any in-depth profiling, running on a setup where tensorflow was locating the devices GPU led to exponentially faster training times.
* Complicated models do not require large amounts of code. Compared to other professional software applications that I have worked on, my diffusion model has a shockingly small number of lines of code. What is much more important here is understanding the concepts, math, and model design. Once the architecture is clear, libraries like tensorflow have great abstractions that make implementation fairly straightforward.
* Data makes all the difference. At first, I played around with training my model on only small pieces of my dataset, such as one specific interior design type. These smaller datasets of ~1.5k images did not produce any meaningful image generations, even after 50 epochs of training. However, in my final training run I included all ~15k images in my training set, and the model was producing different random noise within 10 epochs, and relatively clear images within 30 epochs. This was surprising to me, because I had assumed that if I only gave images of one particular label then each image would be more similar to each other so the model would have a smaller latent space to represent. However, in practice the volume of the additional images proved far more helpful to the model than limiting the dataset to one specific label.